In [ ]:
#https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?symbol=AAPL"
#https://query2.finance.yahoo.com/v8/finance/chart/AAPL?period1=1704145844&period2=1723236317&interval=1mo

In [4]:
import sys
import os
cwd = os.getcwd()

sys.path.append(cwd.replace('notebooks','')+'code')

In [3]:
from source.scraper import get_StockPrices
import pandas as pd

df = get_StockPrices('NVDA', startSelect='2002-01-02')
df

,adjClose,ticker,quarter_meanADJclose,month_meanADJclose,yearMonthDay
0,0.514411,NVDA,0.448435,0.501850,20020102
1,0.548119,NVDA,0.448435,0.501850,20020103
2,0.532145,NVDA,0.448435,0.501850,20020104
3,0.517851,NVDA,0.448435,0.501850,20020107
4,0.523966,NVDA,0.448435,0.501850,20020108
...,...,...,...,...,...
5693,122.860001,NVDA,118.587568,113.710666,20240815
5694,124.580002,NVDA,118.587568,113.710666,20240816
5695,130.000000,NVDA,118.587568,113.710666,20240819
5696,127.250000,NVDA,118.587568,113.710666,20240820


In [ ]:
from source.scraper import generate_CIK_TICKER


df = generate_CIK_TICKER()

In [7]:
from spark_deltalake.spark_operations import sparkDelta


spark = sparkDelta()
pydf = spark.get_spark_dataframe('SEC_filings', True)

hadoop-azure is available
azure-storage is available


In [8]:
pydf.show()

+--------------------+----------+----------+--------------------+---+--------+----------+----------+---------+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|     accessionNumber|filingDate|reportDate|  acceptanceDateTime|act|    form|fileNumber|filmNumber|    items|   size|isXBRL|isInlineXBRL|     primaryDocument|primaryDocDescription|             fileURL|yearMonthDay|ticker|
+--------------------+----------+----------+--------------------+---+--------+----------+----------+---------+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|0000315066-14-004209|2014-11-10|          |2014-11-10T10:52:...| 34|SC 13G/A| 005-13157| 141207194|         |   9330|     0|           0|          filing.txt|                     |https://www.sec.g...|    20141110|   AOS|
|0001193125-14-393314|2014-11-03|2014-09-30|2014-11-03T11:39:...| 34|    10-Q| 001-00475| 141188405|        

In [54]:
ticker = 'AOS'
query = ' '.join(
    (
    "WITH CTE AS (SELECT * FROM SEC_filings",
    f'WHERE ticker = \'{ticker}\')',
    'SELECT ticker, formatTime(MAX(yearMonthDay)) AS max_time, formatTime(MIN(yearMonthDay)) AS min_time FROM CTE',
    'GROUP BY ticker'
    )
 )


def formatTime(timestr):    
    year = timestr[:4]
    month = timestr[4:6]
    day = timestr[6:]
    
    return '-'.join((year, month, day))


spark.spark.udf.register("formatTime", formatTime)

dfspark = spark.spark.sql(query).toPandas()

In [55]:
dfspark

,ticker,max_time,min_time
0,AOS,2024-08-16,2012-12-31


In [ ]:
spark.sparkStop()

In [ ]:
from source.scraper import get_spy500_formWiki


wiki = get_spy500_formWiki()

In [ ]:
get_companyfacts('CIK0000320193')

In [ ]:
from source.scraper import get_companyfacts
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
mergedDF = get_companyfacts('CIK0001045810') #2016-07-31

#mergedDF['fy'] = mergedDF['fy'].replace('',"")
#mergedDF['fy'] = mergedDF['fy'].astype(str).str[:4]
#mergedDF['fy'] = mergedDF['fy'].replace(-1, None)

mergedDF

In [ ]:
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm


https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""

In [ ]:
from source.scraper import generate_CIK_TICKER

sec_ticker = generate_CIK_TICKER()
sec_ticker

In [ ]:
from source.scraper import get_SEC_filings


ticker = 'AAPL'

filings = get_SEC_filings('CIK0000320193', 'AAPL')

filings